In [1]:
import csv
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, QuantoConfig
from speculative_decoding import speculative_decoding
HF_CACHE_LOCATION = "/data/shk148/models/opt/cache"
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

ModuleNotFoundError: No module named 'torch'

In [ ]:
draft_models = [
    "facebook/opt-6.7b",
    "facebook/opt-2.7b",
    "facebook/opt-1.3b",
    "facebook/opt-350m",
    "facebook/opt-125m",
    "facebook/opt-13b",
    "facebook/opt-30b",
]

model_size = {
    "facebook/opt-125m": 125 * 1,
    "facebook/opt-350m": 350 * 1,
    "facebook/opt-1.3b": 1.3 * 1000,
    "facebook/opt-2.7b": 2.7 * 1000,
    "facebook/opt-6.7b": 6.7 * 1000,
    "facebook/opt-13b": 13 * 1000,
    "facebook/opt-30b": 30 * 1000,
}
target_checkpoint = "facebook/opt-350m"
quantization_config = QuantoConfig(weights="int4")
torch.manual_seed(1)

top_k = 20
top_p = 0.9
num_tokens = 20
lookahead = 4


target_model = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b",quantization_config=quantization_config,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m", trust_remote_code=True)

: 

In [ ]:
import gc


def evaluate(assistant_checkpoint, prompt):
    # Perform inference and measure latency
    total_time = 0.0
    draft_model = AutoModelForCausalLM.from_pretrained(assistant_checkpoint, cache_dir=HF_CACHE_LOCATION, device_map="auto", quantization_config=quantization_config)    
    with torch.no_grad():
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(torch_device)
        start_time = time.time()
        output = speculative_decoding(input_ids,draft_model,target_model,num_tokens,tokenizer,lookahead,top_k=top_k,top_p=top_p,)
        total_time += time.time() - start_time
        del output
    torch.cuda.empty_cache()
    del draft_model
    gc.collect()
    torch.cuda.empty_cache()
    return (total_time)

: 

In [ ]:
results = dict()
for model in draft_models:
    temp_result = 0.0
    with open("../data/gpt_prompts.csv", "+r", newline="") as f:
        reader = csv.reader(f)
        for row in reader:
            temp_result = temp_result + evaluate(model, row[0])
        results[model_size[model]] = temp_result

: 